In [1]:
# Structred Data
import numpy as np
import pandas as pd
import torch

# System Libraries
import os
import librosa
import librosa.display
import soundfile as sf

# Garbage Collection
import gc

# Visualization
from IPython.display import Audio, display
import matplotlib.pyplot as plt

#### Set Paths

In [2]:
%cd ..
DIRECTORY_PATH = os.getcwd()

d:\ML\Real-Time-Speech-Recognition-and-Translation-


In [32]:
# Paths
VALIDATED_DATA_PATH = r"\Common_Voice\validated.tsv"
VALIDATED_SENTENCES_PATH = r"\Common_Voice\unvalidated_sentences.tsv"
CLIP_DURATION_PATH = r"\Common_Voice\clip_durations.tsv"
CLIPS_PATH = r"\Common_Voice\clips\\"
CLIPS_WAV_PATH = r"\Common_Voice\clips_wav"

# Constants
NYQUIST_SAMPLING_RATE = 16000

In [58]:
def getDurations(paths):
    duratios = []
    for path in paths:
        audio, sr = librosa.load(DIRECTORY_PATH + CLIPS_PATH + path)
        duratios.append(librosa.get_duration(y=audio, sr=sr))
    return duratios

def getMaxDuration(dataframe):
    #Get Longest Audio Clip
    MAX_AUDIO_DURATION = max(dataframe["duration"])
    LongestAudio = dataframe[dataframe['duration'] == MAX_AUDIO_DURATION]
    PATH_OF_LONGEST_AUDIO = DIRECTORY_PATH + CLIPS_PATH + LongestAudio["path"].values[0]
    SENTENCE_OF_LONGEST_AUDIO = LongestAudio["sentence"].values[0]

    # Load Audio
    wavform, sample_rate = librosa.load(PATH_OF_LONGEST_AUDIO)
    FRAME_LENGTH_OF_LONGEST_AUDIO = len(wavform)
    return MAX_AUDIO_DURATION, PATH_OF_LONGEST_AUDIO, FRAME_LENGTH_OF_LONGEST_AUDIO

def PreprocessAudios(paths, longestAudioFrameLength):
    # Add Padding
    MAXPADDING = longestAudioFrameLength
    for path in paths:
        wavform, sr = librosa.load(DIRECTORY_PATH + CLIPS_PATH + path)
        padding = [0] * (MAXPADDING - len(wavform))
        wavform = np.concatenate((wavform, padding))
        wavform = librosa.resample(wavform, orig_sr=sr, target_sr=NYQUIST_SAMPLING_RATE)
        wavform = librosa.util.normalize(wavform)
        sf.write(DIRECTORY_PATH + CLIPS_WAV_PATH + "\\" + path[:-4] + ".wav", wavform, sr, format='wav')

def getMFCCs(paths):
    wavform, sr = librosa.load(DIRECTORY_PATH + CLIPS_WAV_PATH + "\\" + paths[0])
    mfccs = librosa.feature.mfcc(y=wavform, n_mfcc=13, sr=sr, n_fft=2048, hop_length=512)
    mfccs = mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1])
    for path in paths[1:]:
        wav , sr = librosa.load(DIRECTORY_PATH + CLIPS_WAV_PATH + "\\" + path)
        mfcc = librosa.feature.mfcc(y=wav, n_mfcc=13, sr=sr, n_fft=2048, hop_length=512)
        mfccs = np.concatenate((mfccs, mfcc.reshape(1, mfcc.shape[0], mfcc.shape[1])), axis=0)
    return mfccs

def getTokenizedSentences(sentences):
    # Start Code (Ahmad Elsayed)

    # End Code
    pass

In [38]:
wavform, sr = librosa.load("D:\ML\Real-Time-Speech-Recognition-and-Translation-\Common_Voice\clips_wav\common_voice_en_40189607.wav")
mfcc = librosa.feature.mfcc(y=wavform, n_mfcc=13, sr=sr, n_fft=2048, hop_length=512)
mfccs = mfcc.reshape(1, mfcc.shape[0], mfcc.shape[1])
print(mfccs.shape)


(1, 13, 334)


In [14]:
print(valid_clips_duration.columns)[["path", "sentence", "up_votes", "duration"]]

Index(['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain',
       'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant',
       'locale', 'segment', 'duration'],
      dtype='object')


In [33]:
unfiltered_dataset = pd.read_csv(DIRECTORY_PATH + VALIDATED_DATA_PATH , sep='\t')[["path", "sentence", "up_votes"]]
durations = getDurations(unfiltered_dataset["path"].tolist())
unfiltered_dataset["duration"] = durations
longestAudioDuration ,  longestAudioPath, longestAudioFrameLength = getMaxDuration(unfiltered_dataset)
PreprocessAudios(unfiltered_dataset["path"].tolist(), longestAudioFrameLength)
unfiltered_dataset['path'] = unfiltered_dataset['path'].str.replace('.mp3', '.wav', regex=False)

# For you Ahmad Elsayed uses the following sentences in the following line
sentences = unfiltered_dataset["sentence"].tolist()